In this notebook we learn about one of the most important methods in Pandas: GroupBy. It is a very powerful method that can be used to give us insight into a variety of data. This notebook closely follows the tutorial from RealPython (https://realpython.com/pandas-groupby/) and should be an addition to the content we discussed in the live-session.


In [75]:
import numpy
import requests
import pandas as pd

download_url = "https://raw.githubusercontent.com/rashida048/Datasets/master/phone_data.csv"
target_csv_path = "phone_data.csv"

response = requests.get(download_url)
response.raise_for_status()    # Check that the request was successful
with open(target_csv_path, "wb") as f:
    f.write(response.content)
print("Download ready.")

Download ready.


In [8]:
df = pd.read_csv("phone_data.csv")
df.head()

,index,date,duration,item,month,network,network_type
0,0,15/10/14 06:58,34.429,data,2014-11,data,data
1,1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile
3,3,15/10/14 14:48,4.000,call,2014-11,Tesco,mobile
4,4,15/10/14 17:27,4.000,call,2014-11,Tesco,mobile


In [9]:
dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}
df_us = pd.read_csv(
    "groupby-data/legislators-historical.csv",
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "last_name"],
    parse_dates=["birthday"]
)

df_us.head()

,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN


We see the names of congress members with infos such as birthday party or state (with some old ones in there). How do we get the number of congressional members per state over the whole time period? Entry groupby:

In [17]:
count_by_state = df_us.groupby(["state", "gender"],)["last_name"].count()
count_by_state.head(15)
#as_index=False more closely resembles SQL: Gives back DataFrame with RangeIndex; if true it returns Saries with MultiIndex
#if you use .count() you exclude poosible NaN values; .size() would include those

state  gender
AK     F           0
       M          16
AL     F           3
       M         203
AR     F           5
       M         112
AS     F           0
       M           2
AZ     F           3
       M          45
CA     F          23
       M         338
CO     F           3
       M          87
CT     F           6
Name: last_name, dtype: int64

In [23]:
by_state = df_us.groupby("state")
by_state.head()
#print(by_state)

,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11116,Frazer,Victor,1943-05-24,M,rep,VI,Independent
11338,Underwood,Robert,1948-07-13,M,rep,GU,Democrat
11752,Christensen,Donna,1945-09-19,F,rep,VI,Democrat
11755,Faleomavaega,Eni,1943-08-15,M,rep,AS,Democrat


Groupby operations are often splitted into three steps:

1. **Split**: Split your data into different categories based on some criteria

2. **Apply**: Apply some aggregation operations (such as sum, mean, ...) to the different groups

3. **Combine**: Combine the different results back into the original data frame to get a new result

<img src="../images/.png"

The split operation is lazily implemented in pandas; the groupby operation itself does not do anything we can directly see, but prepares a groupby object which we can further use. We can for example see the different categories by iterating over the groupby object:

In [25]:
for state, frame in by_state:
    print(f"First 2 entries for {state!r}")
    print("---------------")
    print(frame.head(2), end="\n\n")

First 2 entries for 'AK'
---------------
     last_name first_name   birthday gender type state        party
6619    Waskey      Frank 1875-04-20      M  rep    AK     Democrat
6647      Cale     Thomas 1848-09-17      M  rep    AK  Independent

AK
First 2 entries for 'AL'
---------------
    last_name first_name   birthday gender type state       party
912   Crowell       John 1780-09-18      M  rep    AL  Republican
991    Walker       John 1783-08-12      M  sen    AL  Republican

AL
First 2 entries for 'AR'
---------------
     last_name first_name   birthday gender type state party
1001     Bates      James 1788-08-25      M  rep    AR   NaN
1279    Conway      Henry 1793-03-18      M  rep    AR   NaN

AR
First 2 entries for 'AS'
---------------
          last_name first_name   birthday gender type state     party
10797         Sunia       Fofó 1937-03-13      M  rep    AS  Democrat
11755  Faleomavaega        Eni 1943-08-15      M  rep    AS  Democrat

AS
First 2 entries for 'AZ'


3398      Lane     Joseph 1801-12-14      M  sen    OR  Democrat

OR
First 2 entries for 'PA'
---------------
   last_name first_name   birthday gender type state                party
4     Clymer     George 1739-03-16      M  rep    PA                  NaN
19    Maclay    William 1737-07-20      M  sen    PA  Anti-Administration

PA
First 2 entries for 'PI'
---------------
             last_name first_name   birthday gender type state party
6835            Ocampo      Pablo 1853-01-25      M  rep    PI   NaN
6939  Legarda Y Tuason     Benito 1853-09-27      M  rep    PI   NaN

PI
First 2 entries for 'PR'
---------------
      last_name first_name   birthday gender type state       party
6426    Degetau   Federico 1862-12-05      M  rep    PR  Republican
6811  Larrinaga      Tulio 1847-01-15      M  rep    PR    Unionist

PR
First 2 entries for 'RI'
---------------
    last_name first_name   birthday gender type state       party
61   Bradford    William 1729-11-04      M  sen    RI  F

The groupby object is also in some way a dictionary with the group names as keys and the group labels as values (if we call the .groups method on it). So we can index it as a normal dictionary when we call its groups:

In [66]:
dir(by_state)

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accessors',
 '_agg_examples_doc',
 '_agg_general',
 '_agg_py_fallback',
 '_aggregate_frame',
 '_aggregate_item_by_item',
 '_aggregate_with_numba',
 '_apply_allowlist',
 '_apply_filter',
 '_apply_to_column_groupbys',
 '_bool_agg',
 '_cache',
 '_can_use_transform_fast',
 '_choose_path',
 '_concat_objects',
 '_constructor',
 '_cumcount_array',
 '_cython_agg_general',
 '_cython_transform',
 '_define_paths',
 '_dir_additions',
 '_dir_deletions',
 '_fill',
 '_get_cythonized_res

As you see, the groupby object has a lot of methods! In general, you can categorize them into one of the following:

1. **Aggregation/reductions methods**: Squashes a lot of data points into a single number (e.g. mean, median, sum, count)

2. **Filter methods** (including .filter()): Gives back a subset of the original DataFrame, either columns or rows based on some boolean expression

3. **Tranformation methods**: Transforms data in the DataFrame (e.g. unit conversion), but leaves the shape of the DataFrame untouched

4. **Meta methods**: Do not change the original dataframe, but give you information about the outcome of the grouping/splitting process (e.g. .groups, get.group, ...)

5. **Plotting methods**: Similar to normal Pandas plotting, but gives back multiple subplots based on the grouping.

Here a visual intituition about those categories:

<img src="../images/groupby_methods_overview.png"

In [34]:
by_state.groups["AK"]

Int64Index([ 6619,  6647,  7442,  7501,  8039,  8236,  8877,  9819,  9951,
             9985, 10082, 10108, 10325, 11262, 11386, 11734],
           dtype='int64')

In [35]:
by_state.get_group("PA") #same as df_us.loc[df_us["state"] == "PA"]

,last_name,first_name,birthday,gender,type,state,party
4,Clymer,George,1739-03-16,M,rep,PA,NaN
19,Maclay,William,1737-07-20,M,sen,PA,Anti-Administration
21,Morris,Robert,1734-01-20,M,sen,PA,Pro-Administration
27,Wynkoop,Henry,1737-03-02,M,rep,PA,NaN
38,Jacobs,Israel,1726-06-09,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11891,Brady,Robert,1945-04-07,M,rep,PA,Democrat
11932,Shuster,Bill,1961-01-10,M,rep,PA,Republican
11945,Rothfus,Keith,1962-04-25,M,rep,PA,Republican
11959,Costello,Ryan,1976-09-07,M,rep,PA,Republican


In the apply step, we choose some specific operation (aggregation, transformation, filtration) and apply it to every data subset that we produced during splitting:

In [40]:
#get first tuple from iterator groupby object:
state, frame = next(iter(by_state))
state
frame.head(5)

,last_name,first_name,birthday,gender,type,state,party
6619,Waskey,Frank,1875-04-20,M,rep,AK,Democrat
6647,Cale,Thomas,1848-09-17,M,rep,AK,Independent
7442,Grigsby,George,1874-12-02,M,rep,AK,NaN
7501,Sulzer,Charles,1879-02-24,M,rep,AK,NaN
8039,Sutherland,Daniel,1869-04-17,M,rep,AK,Republican


In [41]:
#apply stage applied to one of the DataFrames containing a data subset:
frame["last_name"].count()

16

Combine then only combines all the results from the different subsets and combines them

So far, we grouped based on a string, eg df.groupby("state"). But we could also use different arguments, for example a list of columns instead of a single one, a dict or PandasSeries or a NumPy array or a PandasIndex (or array-like objects of the last two).

In [43]:
df_air = pd.read_csv(
    "groupby-data/airqual.csv",
    parse_dates=[["Date", "Time"]],
    na_values=[-200],
    usecols=["Date", "Time", "CO(GT)", "T", "RH", "AH"]
).rename(
    columns={
        "CO(GT)": "co",
        "Date_Time": "tstamp",
        "T": "temp_c",
        "RH": "rel_hum",
        "AH": "abs_hum",
    }
).set_index("tstamp")

In [44]:
df_air.head()

,co,temp_c,rel_hum,abs_hum
tstamp,,,,
2004-03-10 18:00:00,2.6,13.6,48.9,0.758
2004-03-10 19:00:00,2.0,13.3,47.7,0.726
2004-03-10 20:00:00,2.2,11.9,54.0,0.750
2004-03-10 21:00:00,2.2,11.0,60.0,0.787
2004-03-10 22:00:00,1.6,11.2,59.6,0.789


We could for example use the index of timestamps to get out the corresponding weekdays via the .day_name() method, creating a PandasIndex containing strings (an "array-like object" which we can use with groupby)

In [47]:
weekdays = df_air.index.day_name()
type(weekdays)
weekdays

Index(['Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       ...
       'Monday', 'Monday', 'Monday', 'Monday', 'Monday', 'Monday', 'Monday',
       'Monday', 'Monday', 'Monday'],
      dtype='object', name='tstamp', length=9357)

In [50]:
df_air.groupby(weekdays)["co"].mean() 
#splitting based on artifical column created by us

tstamp
Friday       2.543
Monday       2.017
Saturday     1.861
Sunday       1.438
Thursday     2.456
Tuesday      2.382
Wednesday    2.401
Name: co, dtype: float64

Same can be done for grouping by hour of the day for each weekday, enabling a finer split:

In [57]:
hr = df_air.index.hour
type(hr) #Pandas Int64 Index object
hr
df_air.groupby([weekdays, hr])["co"].mean().rename_axis(["dow", "hr"])

dow        hr
Friday     0     1.936
           1     1.609
           2     1.172
           3     0.887
           4     0.823
                 ...  
Wednesday  19    4.147
           20    3.845
           21    2.898
           22    2.102
           23    1.938
Name: co, Length: 168, dtype: float64

In [65]:
#turn numerical into categorical variable with cut
#then use the resulting Series object as argument for groupby
bins = pd.cut(df_air["temp_c"], bins=3, labels = ("cool", "warm", "hot"))
bins
type(bins)
df_air.groupby(bins).agg("mean")
df_air.groupby(bins).agg(["mean", "median"])
df_air[["rel_hum", "abs_hum"]].groupby(bins).agg(["mean", "median"])

rel_hum        abs_hum       
          mean median    mean median
temp_c                              
cool    57.651   59.2   0.666  0.658
warm    49.383   49.3   1.183  1.145
hot     24.994   24.1   1.293  1.274

*Add-on*: Some advanced features of groupby using a news dataset

In [68]:
import datetime as dt
import pandas as pd

def parse_millisecond_timestamp(ts: int) -> dt.datetime:
    """Convert ms since Unix epoch to UTC datetime instance."""
    return dt.datetime.fromtimestamp(ts / 1000, tz=dt.timezone.utc)

df_news = pd.read_csv(
    "groupby-data/news.csv",
    sep="\t",
    header=None,
    index_col=0,
    names=["title", "url", "outlet", "category", "cluster", "host", "tstamp"],
    parse_dates=["tstamp"],
    date_parser=parse_millisecond_timestamp,
    dtype={
        "outlet": "category",
        "category": "category",
        "cluster": "category",
        "host": "category",
    },
)

In [69]:
df_news.iloc[0]

title       Fed official says weak data caused by weather,...
url         http://www.latimes.com/business/money/la-fi-mo...
outlet                                      Los Angeles Times
category                                                    b
cluster                         ddUyU0VZz0BRneMioxUPQVP6sIxvM
host                                          www.latimes.com
tstamp                       2014-03-10 16:52:50.698000+00:00
Name: 1, dtype: object

With the concept of lamda function, we can answer more sophisticated questions not only based on numerical data but also based on textual information. For example: Which of the newspapers talk more about democrats and which about republicans?

In [83]:
df.groupby("outlet", sort=False)["title"].apply(
    lambda select: select.str.contains("Senate").sum()).nlargest(5)

outlet
Reuters                    11
GlobalPost                  8
Law360 \(subscription\)     7
Businessweek                6
Los Angeles Times           5
Name: title, dtype: int64

In [74]:
df.groupby("outlet", sort=False)["title"].apply(
    lambda select: sele.str.contains("Wall Street").sum()).nlargest(5)

outlet
Proactive Investors USA \& Canada    69
Economic Times                       36
RTT News                             34
Business Standard                    32
Independent Online                   32
Name: title, dtype: int64

To look at what is going on, we can again iterate over the Pandas GroupBy interator and look at a single tuple corresponding to the first group:

In [84]:
title, sele = next(iter(df.groupby("outlet", sort=False)["title"]))
title
sele
type(sele) #series object, not DataFrame since we called only one column

pandas.core.series.Series

In [91]:
#construct boolean mask by invoking string methods
sele.str.contains("Senate")

1         False
486       False
1124      False
1146      False
1237      False
          ...  
421547    False
421584    False
421972    False
422226    False
422905    False
Name: title, Length: 1976, dtype: bool

In [92]:
sele.str.contains("Senate").sum() #False=0, True=1

5

**Conclusion**: .groupby() works by the split-apply-combine paradigm and can use many different input types as long as they can be read as a sequence of labels to perform the grouping/splitting on.